In [ ]:
##
#Import Packages
import numpy as np
import nibabel as nib
import sys
from scipy.optimize import curve_fit

In [ ]:
##
#Define Path to Code database
DirPath = '/PATH/TO/bin/'

#Define Path to Example Data
DataPath = '/PATH/TO/ExampleData/'

#Define Output Path
OutputPath = '/PATH/TO/Output/'

In [ ]:
##
#Import Custom Functions
sys.path.append(DirPath)
from ImportData import *
from FreedAnalytical import *

In [ ]:
##
#Load Data
    # Data - DW-SSFP Data (4D)
    # T1Map - T1 Map (ms, 3D)
    # T2Map - T2 Map (ms, 3D)
    # B1Map - B1 Map (Normalised, 3D)
    # Mask - Mask (Binary, 3D)
    # noisefloor - noisefloor estimate (1xn)
    # bvecs - bvectors (3xn)
    # FlipAngles - Flip Angles (degrees) (1xn)
    # tau - Diffusion Gradient Duration (seconds) (1xn), 
    # G - Diffusion Gradient Duration (G/cm - Equivalent to mT/m Divided by 10) (1xn)
    # TRs - Repetition Times (seconds) (1xn)
    # b0s - Array Defining b0 Locations (b0 = 1, dwi = 0) (1xn)
Data, T1Map, T2Map, B1Map, Mask, noisefloor, bvecs, FlipAngles, tau, G, TRs, b0s = ImportDataDWSSFP(DataPath)

In [ ]:
##
#Estimate Mean Noisefloor
Noisefloor_Mean = np.mean(noisefloor)

In [ ]:
##
#Normalise Data by S0 (Estimate S0 per Flip Angle) & Perform Noisefloor Correction 

##
#Define unique locations
Values, Index = np.unique(FlipAngles, return_index=True)

##
#Estimate normalised data for each flip angle
for idx, k in enumerate(Index):
    #Calculate theoretical signal amplitude
    b0 = FreedDWSSFP(G[k], tau[k], TRs[k], FlipAngles[k]*B1Map.data, 0, T1Map.data, T2Map.data)
    #Identify S0 (incorporating noisefloor contribution)
    S0 = np.abs(np.mean(Data.data[:,:,:,(b0s == 1) & (FlipAngles == Values[idx])],axis = 3)**2-Noisefloor_Mean**2)**0.5/b0*Mask.data
    #Divide by S0 & Remove NoiseFloor
    Data.data[:,:,:,FlipAngles == Values[idx]] = np.abs(Data.data[:,:,:,FlipAngles == Values[idx]]**2-Noisefloor_Mean**2)**0.5/S0[:,:,:,np.newaxis]

In [ ]:
##
#Perform Parameter Estimation (NLLS)

##
#Initialise Output Array
Tensor = np.zeros((*T1Map.data.shape,6))
TensorSD = np.zeros((*T1Map.data.shape,6))


##
#Define Initial Fitting Point & Bounds (NLLS)
Init = np.array([4, 3, 2, -1, 1, -2])*10**-4
low = np.array([0, 0, 0, -1, -1, -1])*10**-3
high = np.array([1, 1, 1, 1, 1, 1])*10**-3

##
#Perform Parameter Estimation for all Voxels in Dataset
for k in range(Data.data.shape[0]):
    for l in range(Data.data.shape[1]):
        for m in range(Data.data.shape[2]):
            if Mask.data[k,l,m] == 0:
                continue
            #Perform Parameter Estimation
            Tensor[k,l,m,:], pcov = curve_fit(lambda x, *theta: FreedDWSSFPTensor_curve_fit(x, theta, G, tau, TRs, FlipAngles, bvecs, B1Map.data[k,l,m], T1Map.data[k,l,m], T2Map.data[k,l,m]), 1, Data.data[k,l,m,:], p0 = Init, bounds=(low,high), method='trf',maxfev=10**6)
            #Calculate Standard Deviation
            TensorSD[k,l,m,:] = np.sqrt(np.diag(pcov))
    print(k)

In [ ]:
##
#Save Outputs

##
#Tensor Components
nib.save(nib.Nifti1Image(Tensor[:,:,:,0],Data.aff),''.join([OutputPath, 'D11_Mean.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,1],Data.aff),''.join([OutputPath, 'D22_Mean.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,2],Data.aff),''.join([OutputPath, 'D33_Mean.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,3],Data.aff),''.join([OutputPath, 'D12_Mean.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,4],Data.aff),''.join([OutputPath, 'D13_Mean.nii.gz']))
nib.save(nib.Nifti1Image(Tensor[:,:,:,5],Data.aff),''.join([OutputPath, 'D23_Mean.nii.gz']))

##
#Tensor Components Standard Deviation
nib.save(nib.Nifti1Image(TensorSD[:,:,:,0],Data.aff),''.join([OutputPath, 'D11_SD.nii.gz']))
nib.save(nib.Nifti1Image(TensorSD[:,:,:,1],Data.aff),''.join([OutputPath, 'D22_SD.nii.gz']))
nib.save(nib.Nifti1Image(TensorSD[:,:,:,2],Data.aff),''.join([OutputPath, 'D33_SD.nii.gz']))
nib.save(nib.Nifti1Image(TensorSD[:,:,:,3],Data.aff),''.join([OutputPath, 'D12_SD.nii.gz']))
nib.save(nib.Nifti1Image(TensorSD[:,:,:,4],Data.aff),''.join([OutputPath, 'D13_SD.nii.gz']))
nib.save(nib.Nifti1Image(TensorSD[:,:,:,5],Data.aff),''.join([OutputPath, 'D23_SD.nii.gz']))

In [ ]:
##
#Convert the Diffusivity Outputs to Eigenvalue/Eigenvector/FA Outputs in FSL via
#fslmerge -t dti D11_Mean.nii.gz D12_Mean.nii.gz D13_Mean.nii.gz D22_Mean.nii.gz D23_Mean.nii.gz D33_Mean.nii.gz
#fslmaths dti.nii.gz -tensor_decomp dti